# Модель движения машин с визуализацией. Изначально на участке дороги машин нет. Можно задать ограничение скорости посреди дороги

In [1]:
from tkinter import *
import time
import random
import numpy as np
import math

In [2]:
class car():    
    def nearest_cars(self):
        n_h = None
        n_t = None 
        p_h = None 
        p_t = None
        l_next_h = 10000
        l_prev_h = -10000
        l_next_t = 10000
        l_prev_t = -10000
        for car_ in self.cars_list:
            if (self.line == car_.line):
                if car_.x - self.x < 0:
                    if car_.x - self.x > l_prev_h:
                        l_prev_h = car_.x - self.x
                        p_h = car_
                if car_.x - self.x > 0:
                    if car_.x - self.x < l_next_h:
                        l_next_h = car_.x - self.x
                        n_h = car_
            else:
                if car_.x - self.x < 0:
                    if car_.x - self.x > l_prev_t:
                        l_prev_t = car_.x - self.x
                        p_t = car_
                if car_.x - self.x >= 0:
                    if car_.x - self.x < l_next_t:
                        l_next_t = car_.x - self.x
                        n_t = car_
        # теперь краевые случаи
        if n_h == None:
            for car_ in last_c:
                if car_.line == self.line:
                    n_h = car_
        if n_t == None:
            for car_ in last_c:
                if car_.line != self.line:
                    n_t = car_
                    
        if p_h == None:
            for car_ in last_c:
                if car_.line == self.line:
                    p_h = car_
        if p_t == None:
            for car_ in last_c:
                if car_.line != self.line:
                    p_t = car_          
            
        result = [n_h, n_t, p_h, p_t]
        return(result)
    
    def __init__(self, canvas, x0, v0, line, cars_list = None):
        self.canvas = canvas
        self.size = [20, 10]
        self.v = v0
        self.x = x0
        if self.x >= ogr:
            self.v_max = v_max_2
        else:
            self.v_max = v_max
        self.line = line
        self.a = 0
        self.cars_list = cars_list
        if cars_list != None:
            n_cars = self.nearest_cars()
        self.next_here = None # ссылка на следующуу машину в этом ряду
        self.next_there = None # ссылка на следующую машину в соседнем ряду
        self.prev_here = None 
        self.prev_there = None
        
        if cars_list != None:
            if line == 1:
                self.y = 240
            else:
                self.y = 260
            self.id = canvas.create_rectangle(self.x - self.size[0]/2 + 50,
                                              self.y - self.size[1]/2,
                                              self.x + self.size[0]/2 + 50,
                                              self.y + self.size[1]/2,
                                              fill = "red")
        
    def obgon(self):
        if self.line == 1:
            self.line = 2
            dy = 20
            self.y = 260
        else:
            self.line = 1
            dy = -20
            self.y = 240
        self.canvas.move(self.id, 0, dy)
        
        n_cars = self.nearest_cars()
        self.next_here = n_cars[0] 
        self.next_there = n_cars[1]
        self.prev_here = n_cars[2] 
        self.prev_there = n_cars[3]
        
        
    def draw(self):
        #---------------1-------------
        n_cars = self.nearest_cars()
        self.next_here = n_cars[0] 
        self.next_there = n_cars[1]
        self.prev_here = n_cars[2] 
        self.prev_there = n_cars[3]
        
        self_l = self.next_here.x - self.x  # ТУТ ВЫЧИСЛЯЕТСЯ l
        #--------------2-------------
        dx = self.v * t + (self.a * (t**2))/2
        old_x = self.x
        self.x = self.x + dx
        
        if self.x >= ogr:  # Ограничение скорости посреди дороги
            self.v_max = v_max_2
        #--------------3------------
        self.l = self.next_here.x - self.x # ТУТ ВЫЧИСЛЯЕТСЯ l
        if self.l <= l_min + 10:
            if (self.next_there.x - self.x > l_min*2) & (self.x - self.prev_there.x > l_min*2):
                self.obgon()
        
        la = self.l - l_min - 40  # здесь задается расстояние для рассчета ускорения
        anew = la * alpha
        vnew = self.v + self.a * t
        if vnew < 0:
            vnew = 0
        if (vnew > self.v_max):
            vnew = self.v_max
            anew = 0
        if (self.l < l_min):
            vnew = self.next_here.v
            self.l = l_min
            
        self.v = vnew
        self.a = anew  
        #-------------4--------------
        if self.x > 1800:
            ind = self.cars_list.index(self)
            self.cars_list.pop(ind)
        else:
            delta = self.x - old_x 
            self.canvas.move(self.id, delta, 0)
        

In [3]:
def count_L(cars, line): # Функция считает первую машину на наблюдаемом участке
    x = 1000000
    for car_ in cars:
        if car_.x < x and car_.line == line:
            x = car_.x
    return x

In [4]:
def draw_cars(v_max, v_max_2, ogr, l_min, t, alpha):
    tk = Tk()
    tk.title('graph')
    tk.resizable(0, 0)
    canvas = Canvas(tk, width=1900, height=500, highlightthickness=0)
    canvas.pack()

    canvas.create_line(0, 250, 1900, 250)
    canvas.create_line(0, 230, 1900, 230)
    canvas.create_line(0, 270, 1900, 270)
    tk.update()
    
    global last_c
    global first_c
    first_1 = car(canvas, -100, 0, 1)
    first_2 = car(canvas, -100, 0, 2)
    last_1 = car(canvas, 1900, 0, 1)
    last_2 = car(canvas, 1900, 0, 2)
    last_c = [last_1, last_2]
    first_c = [first_1, first_2]
    
    cars = []
    Lines = [1, 2]
    while 1:
        for line in Lines:
            L = count_L(cars, line)
            if L > 60:
                p = random.choice([1, 2, 3, 4])
                if p == 1:
                    car_ = car(canvas, 0, 150, line, cars)
                    cars.append(car_)
        for car_ in cars:
            car_.draw()
        tk.update_idletasks()
        tk.update()
        time.sleep(0.05)

In [5]:
v_max = 200 # максимальная скорость
v_max_2 = 100 # скорость после ограничения
ogr =  1000# координата ограничения скорости
l_min = 20 + 20 # мин расстояние до след. машины
t = 0.1 # время опроса
alpha = 0.3 # коэффициент ускорения

In [6]:
draw_cars(v_max, v_max_2, ogr, l_min, t, alpha)

KeyboardInterrupt: 

------

## Старый код

In [ ]:
tk = Tk()
tk.title('graph')
tk.resizable(0, 0)
canvas = Canvas(tk, width=1900, height=500, highlightthickness=0)
canvas.pack()

canvas.create_line(0, 250, 1900, 250)
canvas.create_line(0, 230, 1900, 230)
canvas.create_line(0, 270, 1900, 270)
tk.update()

In [ ]:
first_1 = car(canvas, -100, 0, 1)
first_2 = car(canvas, -100, 0, 2)
last_1 = car(canvas, 1900, 0, 1)
last_2 = car(canvas, 1900, 0, 2)
last_c = [last_1, last_2]
first_c = [first_1, first_2]

In [ ]:
# задать массив скоростей
arv = []
for i in range(30):
    arv.append(150)

# задать массив координат
'''
while(1):
    ar = []
    for i in range(30):
        ar.append(int(random.uniform(0, 1800)))
    ar = sorted(ar)
    f = 0
    for i in range(29):
        if ar[i + 1] - ar[i] < l_min:
            f = 1     #ЭТО НЕ РАБОТАЕТ НАДО ПОМЕНЯТЬ
    if f == 1:
        continue
    break
'''
ar = []
for i in range(30):
    ar.append((1800/30) * i)
    ar = sorted(ar)

# задать массив полосы
lines = []
for i in range(30):
    lines.append(random.randint(1, 2))

In [ ]:
cars = []
# for i in range(30):
#     car_ = car(canvas, ar[i], arv[i], lines[i], cars)
#     cars.append(car_)
# for car_ in cars:
#     n_cars = car_.nearest_cars()
#     car_.next_here = n_cars[0] # ссылка на следующуу машину в этом ряду
#     car_.next_there = n_cars[1] # ссылка на следующую машину в соседнем ряду
#     car_.prev_here = n_cars[2] 
#     car_.prev_there = n_cars[3]

In [ ]:
Lines = [1, 2]
while 1:
    for line in Lines:
        L = count_L(cars, line)
        if L > 60:
            p = random.choice([1, 2, 3, 4])
            if p == 1:
                car_ = car(canvas, 0, 150, line, cars)
                cars.append(car_)
    for car_ in cars:
        car_.draw()
    tk.update_idletasks()
    tk.update()
    time.sleep(0.05)